In [1]:
import open3d as o3d
import numpy as np
import cv2
import PIL
import matplotlib.pyplot as plt
import imshowpair
import copy
import glob
import jupyternotify
import re
import imutils
import os

%matplotlib inline
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%config IPCompleter.greedy = True

ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)
%autonotify -a 120

plt.rcParams['figure.figsize'] = [30, 15]

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


<IPython.core.display.Javascript object>

# Loading in the Two Camera's RGB-D Images

In [9]:
# For sorting the images
def get_key(fp):
    filename = os.path.splitext(os.path.basename(fp))[0]
    int_part = filename.split()[0]
    int_part = int_part.split('.')[0]
    return int(int_part)
#----------------------------------------------------------
# For loading the images
def load_images(path, is_depth = False):
    frames = []
    if is_depth:
        files = glob.glob (path + "/*.png")
    else:
        files = glob.glob (path + "/*.jpg")
    
    files = sorted(files, key=get_key)
    for myFile in files:
        image = cv2.imread(myFile)
        frames.append (image)
    frames = np.array(frames)
    return frames
#----------------------------------------------------------    

rgb_top_path = '/var/Programs/Data/Cotton Imaging/18_9_top_RTabMap_Images/18_9_top_rgb'
depth_top_path = '/var/Programs/Data/Cotton Imaging/18_9_top_RTabMap_Images/18_9_top_depth'
rgb_bot_path = '/var/Programs/Data/Cotton Imaging/18_9_bot_RTabMap_Images/18_9_bot_left'
depth_bot_path = '/var/Programs/Data/Cotton Imaging/18_9_bot_RTabMap_Images/18_9_bot_depth'

rgb_top_images = load_images(rgb_top_path)
depth_top_images = load_images(depth_top_path, True)
rgb_bot_images = load_images(rgb_bot_path)
depth_bot_images = load_images(depth_bot_path, True)

# Function for Stitching the RGB-D Images of one Camera using SIFT

In [51]:
def stitch_rgbd(rgb_images, depth_images):
    img_1 = rgb_images[0]
    output_rgb = rgb_images[0]
    img_1_depth = depth_images[0]
    output_depth = depth_images[0]
    img1 = cv2.cvtColor(img_1,cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    
    for i in range(1, len(rgb_images)):
        img_2 = rgb_images[i]
        img_2_depth = depth_images[i]
        img2 = cv2.cvtColor(img_2,cv2.COLOR_BGR2GRAY)
        
        kp1, des1 = sift.detectAndCompute(img1[:,:img_2.shape[1]],None)
        kp2, des2 = sift.detectAndCompute(img2,None)
        
        # Brute-Force Matcher using L2-norm as measurement tool
        bf = cv2.BFMatcher()
        
        # Finds the k best matches for each descriptor from a query set.
        matches = bf.knnMatch(des1,des2, k=2)
        
        # Applying the Ratio Test by D. Lowe
        if len(matches) > 6000:
            ratio = 0.8
        elif len(matches) <= 6000 and len(matches) >4650:
            ratio = 0.5
        else:
            ratio = 0.7
        
        good = []
        for m in matches:
            if (m[0].distance < ratio*m[1].distance):
                good.append(m)
        matches = np.asarray(good)

        if (len(matches[:,0]) >= 4):
            src = np.float32([ kp1[m.queryIdx].pt for m in matches[:,0] ]).reshape(-1,1,2)
            dst = np.float32([ kp2[m.trainIdx].pt for m in matches[:,0] ]).reshape(-1,1,2)
            H, masked = cv2.findHomography(src, dst, cv2.RANSAC, 5.0)
        else:
            print(f"For image #{i}")
            raise AssertionError('Can’t find enough keypoints.')
        
        print(f"Image #{i}")
        # RGB Stitching
        dst = cv2.warpPerspective(img_1,H,((img_1.shape[1]+ img_2.shape[1]), img_2.shape[0])) #wraped image
        dst[0:img_2.shape[0], 0:img_2.shape[1]] = img_2 #stitched image

        # Depth Stitching
        dst_depth = cv2.warpPerspective(img_1_depth,H,((img_1.shape[1] + img_2.shape[1]), img_2.shape[0])) #wraped image
        dst_depth[0:img_2.shape[0], 0:img_2.shape[1]]= img_2_depth #stitched image
        
        if i % 3 == 0 :
            dst_gray = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
            indices = np.argwhere(dst_gray[360,:] == 0)[30]
            output_rgb = dst[:,:indices[0]]
            output_depth = dst_depth[:,:indices[0]]
        else:
            output_rgb = dst
            output_depth = dst_depth
            
        if i != len(rgb_images) - 1:
            # Just looking at the previous image
            img_1 = output_rgb
            img_1_depth = output_depth
            img1 = cv2.cvtColor(img_1,cv2.COLOR_BGR2GRAY)
        
    return output_rgb, output_depth


## Testing Function

In [52]:
top_rgb, top_depth = stitch_rgbd(rgb_top_images[:62], depth_top_images[:62])

cv2.imwrite('top_rgb.jpg', top_rgb)
cv2.imwrite('top_depth.png', top_depth)

Image #1
Image #2
Image #3
Image #4
Image #5
Image #6
Image #7
Image #8
Image #9
Image #10
Image #11
Image #12
Image #13
Image #14
Image #15
Image #16
Image #17
Image #18
Image #19
Image #20
Image #21
Image #22
Image #23
Image #24
Image #25
Image #26
Image #27
Image #28
Image #29
Image #30
Image #31
Image #32
Image #33
Image #34
Image #35
Image #36
Image #37
Image #38
Image #39
Image #40
Image #41
Image #42
Image #43
Image #44
Image #45
Image #46
Image #47
Image #48
Image #49
Image #50
Image #51
Image #52
Image #53
Image #54
Image #55
Image #56
Image #57
Image #58
Image #59
Image #60
Image #61


True

<IPython.core.display.Javascript object>

In [ ]:
or i == len(rgb_images) - 1